# Metaheuristics for Optimization/Decision Problems


1. **Solution Representation**:
   The solution representation for this problem can be a set of INGREDIENTS that will be included in the pizza. Each ingredient appears only once in the set, and the order of ingredients doesn't matter. This set represents the ingredients to be included in the pizza.
   I: Set of INGREDIENTS.

2. **Neighborhood/Mutation**:
   - **Add/Remove Ingredient**: With a certain probability, randomly select an ingredient from the set of all possible INGREDIENTS. If the selected ingredient is not already in the solution, add it. If the selected ingredient is already in the solution, remove it.

3. **Crossover Functions**:
   For crossover, we can use a one-point crossover approach. However, since we're dealing with sets, the concept of crossover becomes a bit different. We can combine INGREDIENTS from both parent sets to create two child sets, ensuring that there are no duplicate INGREDIENTS in the offspring.

4. **Hard Constraints**:
   There's one hard constraint for this problem:
   - **Non-Empty Solution**: Ensure that the solution set contains at least one ingredient. If, after mutation or crossover, the solution becomes empty, add a randomly selected ingredient from the set of all possible INGREDIENTS to the solution to maintain its non-emptiness. This ensures that there is always at least one ingredient on the pizza, as an empty pizza would not attract any clients.

5. **Evaluation Functions**:
   The evaluation function calculates the fitness of a solution by counting how many clients will visit the pizzeria with that solution. To do this, iterate over each client and check if all the INGREDIENTS they like are present in the solution and none of the ingredients they dislike are present. Increment a counter for each client that meets these conditions, and return the total count as the fitness score for the solution.

In [85]:
import random
import copy
import numpy as np
import warnings
warnings.filterwarnings('ignore')


def read_input_file(file_path) ->tuple[int, list]:
    """Reads the input

    Args:
        file_path (str): File path to the input file

    Returns:
        tuple: Number of clients, list of tuples (likes, dislikes)
    """
    with open(file_path, 'r') as file:
        # first line, 1 <= C <= 10^5 potential clients
        num_clients = int(file.readline().strip())

        # following 2*C lines, list of tuples (likes, dislikes)
        client_preferences = []
        for _ in range(num_clients):
            likes = file.readline().strip().split()[1:] # 1 ≤ L ≤ 5 likes
            dislikes = file.readline().strip().split()[1:] # 0 ≤ D ≤ 5 dislikes
            client_preferences.append((likes, dislikes))
    return num_clients, client_preferences

# Input
file_path = 'example1.in'

num_clients, CLIENT_PREFERENCES = read_input_file(file_path)
print("Number of Clients:", num_clients)
for likes, dislikes in CLIENT_PREFERENCES:
    print("Likes:", likes)
    print("Dislikes:", dislikes)

Number of Clients: 3
Likes: ['cheese', 'peppers']
Dislikes: []
Likes: ['basil']
Dislikes: ['pineapple']
Likes: ['mushrooms', 'tomatoes']
Dislikes: ['basil']


In [86]:
# Returns a list with all the ingredients
def get_all_ingredients() -> set[str]:
    """Return a set with all the ingredients

    Returns:
        set[str]: All the ingredients.
    """
    
    all_ingredients = set()
    for client in CLIENT_PREFERENCES:
        all_ingredients.update(client[0])
        all_ingredients.update(client[1])
    return all_ingredients

def random_solution() -> set[str]:
    """Return a random solution
    Returns:
        set[str]: A random solution.
    """
    return set(random.sample(INGREDIENTS, random.randint(1, len(INGREDIENTS))))

# Scoring
def evaluate_solution(state) -> int:
    """Evaluate a state

    Args:
        solution (set): Current state

    Returns:
        int: Score
    """
    score = 0
    for client in CLIENT_PREFERENCES:
        likes = set(client[0])
        dislikes = set(client[1])
        # all the ingredients they like are on the pizza, none of the ingredients they dislike are on the pizza
        if likes.issubset(state) and not any(dislike in state for dislike in dislikes):
            score += 1
    return score


INGREDIENTS = get_all_ingredients()

INITIAL_SOLUTION = random_solution()
INITIAL_SCORE = evaluate_solution( INITIAL_SOLUTION)
print("Initial Solution:", INITIAL_SOLUTION)
print("Score:", INITIAL_SCORE)

Initial Solution: {'mushrooms', 'cheese', 'basil'}
Score: 1


In [87]:
# Submission - Number of ingredients on the pizza followed by unordered list of them, without repetitions
output = f"{len(INITIAL_SOLUTION)} {' '.join(INITIAL_SOLUTION)}"
print(output)

3 mushrooms cheese basil


In [88]:
# MUTATION AND CROSSOVER
def mutate_solution(state):
    """Mutation and crossover

    Args:
        state: Current state

    Returns:
        list: Next state
    """
    mutated_solution = state.copy()
    mutated_solution = list(state)  # Ensure mutated_solution is a list
    # Remove or add an ingredient randomly
    if random.random() < 0.5:  # Remove
        if mutated_solution:
            mutated_solution.remove(random.choice(mutated_solution))
    else:  # Add
        available_ingredients = [i for i in INGREDIENTS if i not in mutated_solution]
        if available_ingredients:
            ingredient_to_add = random.choice(available_ingredients)
            mutated_solution.append(ingredient_to_add)

    if mutated_solution == []:
        return mutate_solution(mutated_solution)
    return mutated_solution

In [89]:
mutated_solution = mutate_solution(INITIAL_SOLUTION)
mutated_score = evaluate_solution(mutated_solution)
print("Mutated Solution:", mutated_solution)
print("Score:", INITIAL_SCORE)

Mutated Solution: ['mushrooms', 'cheese', 'basil', 'pineapple']
Score: 1


## Simulated Annealing

In [90]:
# Simulated Annealing
def get_sa_solution(num_iterations, log=False)-> set[str] | list[str]:
    iteration = 0
    temperature = 1000
    state = random_solution() # Best solution after 'num_iterations' iterations without improvement
    score = evaluate_solution(state)

    best_solution = copy.deepcopy(state)
    best_score = score

    print(f"Init Solution:  {best_solution}, score: {best_score}")

    while iteration < num_iterations:
        temperature = temperature * 0.999  # Test with different cooling schedules
        iteration += 1

        neighbor_solution = mutate_solution(best_solution)
        neighbor_score = evaluate_solution( neighbor_solution)
        delta = neighbor_score - score
        if (delta > 0 or random.random() < np.exp(delta/temperature)):
            state = neighbor_solution
            score = neighbor_score

            if log:
                    print(f"Solution:       {best_solution}, score: {best_score},  Temp: {temperature}")

            if (score > best_score):
                best_solution = copy.deepcopy(state)
                best_score = score
                iteration = 0
                if log:
                    print(f"Best Solution:  {best_solution}, score: {best_score},  Temp: {temperature}, Prob: {np.exp(delta/temperature)}")

    print(f"Final Solution: {best_solution}, score: {best_score}")
    return best_solution

In [91]:
print("\nSimulated Annealing:\n")
final_solution = get_sa_solution(10000, False)
print(f"{len(final_solution)} {' '.join(final_solution)}")


Simulated Annealing:

Init Solution:  {'cheese'}, score: 0
Final Solution: ['cheese', 'peppers', 'basil'], score: 2
3 cheese peppers basil
